# البناء باستخدام نماذج عائلة Meta

## المقدمة

سنتناول في هذا الدرس:

- استكشاف النموذجين الرئيسيين من عائلة Meta - Llama 3.1 و Llama 3.2
- فهم حالات الاستخدام والسيناريوهات المناسبة لكل نموذج
- مثال برمجي يوضح الميزات الفريدة لكل نموذج

## عائلة نماذج Meta

في هذا الدرس، سنستعرض نموذجين من عائلة Meta أو ما يسمى بـ "قطيع Llama" - وهما Llama 3.1 و Llama 3.2

تتوفر هذه النماذج بعدة إصدارات ويمكن الحصول عليها من سوق النماذج على Github. لمزيد من التفاصيل حول استخدام نماذج Github لـ [النمذجة الأولية مع نماذج الذكاء الاصطناعي](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst).

إصدارات النماذج:
- Llama 3.1 - 70B Instruct
- Llama 3.1 - 405B Instruct
- Llama 3.2 - 11B Vision Instruct
- Llama 3.2 - 90B Vision Instruct

*ملاحظة: يتوفر Llama 3 أيضاً على Github Models لكنه لن يكون ضمن محتوى هذا الدرس*


## Llama 3.1

بـ 405 مليار معلمة، تندرج Llama 3.1 ضمن فئة نماذج اللغة الكبيرة مفتوحة المصدر.

هذا الإصدار هو تطوير للإصدار السابق Llama 3 من خلال تقديم:

- نافذة سياق أكبر - 128 ألف رمز مقابل 8 آلاف رمز
- حد أقصى أكبر لعدد الرموز الناتجة - 4096 مقابل 2048
- دعم لغات متعددة بشكل أفضل - بسبب زيادة عدد الرموز المستخدمة في التدريب

هذه الميزات تمكّن Llama 3.1 من التعامل مع حالات استخدام أكثر تعقيدًا عند بناء تطبيقات الذكاء الاصطناعي التوليدي، بما في ذلك:
- استدعاء الدوال بشكل مباشر - القدرة على استدعاء أدوات ودوال خارج سير عمل النموذج اللغوي
- أداء أفضل في استرجاع المعلومات المعزز - بفضل نافذة السياق الأكبر
- توليد بيانات اصطناعية - القدرة على إنشاء بيانات فعّالة لمهام مثل التخصيص الدقيق


### استدعاء الدوال الأصلية

تم تحسين Llama 3.1 ليكون أكثر فعالية في استدعاء الدوال أو الأدوات. كما يحتوي على أداتين مدمجتين يمكن للنموذج التعرف على الحاجة لاستخدامهما بناءً على طلب المستخدم. هاتان الأداتان هما:

- **Brave Search** - يمكن استخدامها للحصول على معلومات حديثة مثل حالة الطقس من خلال البحث على الإنترنت
- **Wolfram Alpha** - يمكن استخدامها لإجراء حسابات رياضية معقدة، وبالتالي لن تحتاج لكتابة دوالك الخاصة

يمكنك أيضًا إنشاء أدواتك المخصصة التي يمكن لـ LLM استدعاؤها.

في مثال الكود أدناه:

- نحدد الأدوات المتاحة (brave_search, wolfram_alpha) في رسالة النظام.
- نرسل طلب مستخدم يسأل عن حالة الطقس في مدينة معينة.
- سيستجيب LLM باستدعاء أداة Brave Search وسيبدو الاستدعاء بهذا الشكل `<|python_tag|>brave_search.call(query="Stockholm weather")`

*ملاحظة: هذا المثال يقوم فقط باستدعاء الأداة، إذا كنت ترغب في الحصول على النتائج، ستحتاج إلى إنشاء حساب مجاني في صفحة Brave API وتعريف الدالة بنفسك*


In [ ]:
%pip install azure-core
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import AssistantMessage, SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"
model_name = "meta-llama-3.1-405b-instruct"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)


tool_prompt=f"""
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Environment: ipython
Tools: brave_search, wolfram_alpha
Cutting Knowledge Date: December 2023
Today Date: 23 July 2024

You are a helpful assistant<|eot_id|>
"""

messages = [
    SystemMessage(content=tool_prompt),
    UserMessage(content="What is the weather in Stockholm?"),

]

response = client.complete(messages=messages, model=model_name)

print(response.choices[0].message.content)





### لاما 3.2

على الرغم من أن لاما 3.1 هو نموذج لغوي كبير، إلا أن أحد قيوده هو عدم دعمه للوسائط المتعددة. أي أنه لا يستطيع استخدام أنواع مختلفة من المدخلات مثل الصور كمدخلات وتقديم ردود عليها. هذه القدرة تُعد من الميزات الرئيسية في لاما 3.2. وتشمل هذه الميزات أيضًا:

- دعم الوسائط المتعددة - يمكنه تقييم كل من النصوص والصور كمدخلات
- إصدارات بأحجام صغيرة إلى متوسطة (11B و90B) - هذا يوفر خيارات نشر مرنة،
- إصدارات نصية فقط (1B و3B) - هذا يسمح بنشر النموذج على الأجهزة الطرفية أو المحمولة ويوفر استجابة سريعة

دعم الوسائط المتعددة يُعد خطوة كبيرة في عالم النماذج مفتوحة المصدر. المثال البرمجي أدناه يأخذ صورة ونصًا كمدخلات للحصول على تحليل للصورة من لاما 3.2 90B.

### دعم الوسائط المتعددة مع لاما 3.2


In [ ]:
%pip install azure-core
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import (
    SystemMessage,
    UserMessage,
    TextContentItem,
    ImageContentItem,
    ImageUrl,
    ImageDetailLevel,
)
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"
model_name = "Llama-3.2-90B-Vision-Instruct"


client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(
            content="You are a helpful assistant that describes images in details."
        ),
        UserMessage(
            content=[
                TextContentItem(text="What's in this image?"),
                ImageContentItem(
                    image_url=ImageUrl.load(
                        image_file="sample.jpg",
                        image_format="jpg",
                        detail=ImageDetailLevel.LOW)
                ),
            ],
        ),
    ],
    model=model_name,
)

print(response.choices[0].message.content)

## التعلم لا يتوقف هنا، واصل الرحلة

بعد الانتهاء من هذا الدرس، اطلع على [مجموعة تعلم الذكاء الاصطناعي التوليدي](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst) لمواصلة تطوير معرفتك في الذكاء الاصطناعي التوليدي!



---

**إخلاء المسؤولية**:  
تمت ترجمة هذا المستند باستخدام خدمة الترجمة بالذكاء الاصطناعي [Co-op Translator](https://github.com/Azure/co-op-translator). بينما نسعى لتحقيق الدقة، يرجى العلم أن الترجمات الآلية قد تحتوي على أخطاء أو عدم دقة. يجب اعتبار المستند الأصلي بلغته الأصلية هو المصدر الموثوق. للمعلومات الهامة، يُنصح بالاستعانة بترجمة بشرية احترافية. نحن غير مسؤولين عن أي سوء فهم أو تفسير خاطئ ينشأ عن استخدام هذه الترجمة.
